<a href="https://colab.research.google.com/github/DevendraSiShekhawat/Diabetes-Detection/blob/main/Diabetes_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas scikit-learn tensorflow jupyterlab xgboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.3/383.3 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling 

In [ ]:
!pip install ipykernel==5.5.6
!pip install jupyter-client==5.3.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.29.4
    Uninstalling ipykernel-6.29.4:
      Successfully uninstalled ipykernel-6.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.0 requires ipykernel>=6.5.0, but you have ipykernel 5.5.6 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.1 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Load the dataset
df = pd.read_csv("diabetes_prediction_dataset.csv")

# Perform one-hot encoding for categorical features
categorical_features = ['gender', 'hypertension', 'heart_disease', 'smoking_history']
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Separate features and target variable
X = df.drop(columns=['diabetes'])
y = df['diabetes']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Perform feature selection
selector = SelectKBest(score_func=f_classif, k="all")
X_train_selected = selector.fit_transform(X_train, y_train)
selected_indices = selector.get_support(indices=True)
selected_features_names = X_train.columns[selected_indices]
print("Selected features:", selected_features_names)

# Perform over-sampling using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_selected, y_train)

# Perform under-sampling using RandomUnderSampler
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Define and compile ANN model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_resampled.shape[1],)),  # First hidden layer with ReLU activation
    Dense(128, activation='tanh'),  # Second hidden layer with Tanh activation
    Dense(64, activation='relu'),   # Third hidden layer with ReLU activation
    Dense(1, activation='sigmoid')  # Output layer with Sigmoid activation for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train ANN model with early stopping
history = model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32,
                    validation_split=0.2, callbacks=[early_stopping], verbose=2)

# Evaluate performance
_, ann_accuracy = model.evaluate(X_test_scaled, y_test)
print("ANN Accuracy:", ann_accuracy)


Selected features: Index(['age', 'bmi', 'HbA1c_level', 'blood_glucose_level', 'gender_Male',
       'gender_Other', 'hypertension_1', 'heart_disease_1',
       'smoking_history_current', 'smoking_history_ever',
       'smoking_history_former', 'smoking_history_never',
       'smoking_history_not current'],
      dtype='object')
Epoch 1/100
3661/3661 - 10s - loss: 0.5328 - accuracy: 0.6986 - val_loss: 0.6474 - val_accuracy: 0.7267 - 10s/epoch - 3ms/step
Epoch 2/100
3661/3661 - 8s - loss: 0.3618 - accuracy: 0.8284 - val_loss: 0.5524 - val_accuracy: 0.7045 - 8s/epoch - 2ms/step
Epoch 3/100
3661/3661 - 9s - loss: 0.2534 - accuracy: 0.8814 - val_loss: 0.1659 - val_accuracy: 0.9606 - 9s/epoch - 2ms/step
Epoch 4/100
3661/3661 - 9s - loss: 0.2318 - accuracy: 0.8894 - val_loss: 0.4399 - val_accuracy: 0.7162 - 9s/epoch - 3ms/step
Epoch 5/100
3661/3661 - 7s - loss: 0.2198 - accuracy: 0.8964 - val_loss: 0.0791 - val_accuracy: 0.9828 - 7s/epoch - 2ms/step
Epoch 6/100
3661/3661 - 9s - loss: 0.2102 -